<a href="https://colab.research.google.com/github/thomouvic/txtanalytics/blob/main/tfidf_createvecs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://github.com/thomouvic/txtanalytics/raw/main/txts.zip
!unzip -q txts.zip

--2022-11-14 00:02:50--  https://github.com/thomouvic/txtanalytics/raw/main/txts.zip
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/thomouvic/txtanalytics/main/txts.zip [following]
--2022-11-14 00:02:50--  https://raw.githubusercontent.com/thomouvic/txtanalytics/main/txts.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3274958 (3.1M) [application/zip]
Saving to: ‘txts.zip’

txts.zip            100%[===================>]   3.12M  --.-KB/s    in 0.05s   

2022-11-14 00:02:50 (58.7 MB/s) - ‘txts.zip’ saved [3274958/3274958]

replace txts/03_Jensen_Its-not-Personal_WJS_Vol13_No-2_Fall-2017_

In [3]:
from pathlib import Path

all_txt_files =[]
for file in Path("./txts/").rglob("*.txt"):
  # print(file)
  all_txt_files.append(file.parent / file.name)

n_files = len(all_txt_files)
print(n_files)

130


In [4]:
import pandas as pd

all_txt_files.sort()
all_txt_files[0]

# A: store docnames to a pandas frame, then to a csv file
df_docnames = pd.DataFrame({'docname':all_txt_files})
df_docnames['docid'] = df_docnames.index
df_docnames.to_csv('docnames.csv', index=False)

print(df_docnames)

                                               docname  docid
0    txts/03_Jensen_Its-not-Personal_WJS_Vol13_No-2...      0
1         txts/ACH-ALL-2005-Conference-Proceedings.txt      1
2                                    txts/Abstract.txt      2
3    txts/Arbuckle-2020-How-Can-We-Broaden-and-Dive...      3
4    txts/Arbuckle-Christie-Siemens_IntroductionSRC...      4
..                                                 ...    ...
125  txts/Winter-et-al-2020-Foundations-for-the-Can...    125
126                        txts/eludamos7.1_jensen.txt    126
127                                 txts/hubREADME.txt    127
128                           txts/newDraftVersion.txt    128
129                       txts/project_muse_787134.txt    129

[130 rows x 2 columns]


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

all_docs = [open(f).read() for f in all_txt_files]
vectorizer = TfidfVectorizer(max_df=.65, min_df=1, stop_words={'english'}) # A: removed norm=None
transformed_documents = vectorizer.fit_transform(all_docs) # A: This is a sparse matrix

print(transformed_documents)
# A: In a sparse matrix, we have the rows and cols that have non-zero entries, 
# the numbers are the tfidf of the words. 
# E.g. (0, 6132)	0.0018000350124242881 means that in doc 0, word 6132 
# has tfidf 0.0018000350124242881
# If you want to see what's word 6132, do vectorizer.get_feature_names()[6132]
print(vectorizer.get_feature_names_out())
print(vectorizer.get_feature_names_out()[6132])

  (0, 1845)	0.0034066976223548744
  (0, 1840)	0.0034066976223548744
  (0, 1523)	0.0027646746887672454
  (0, 6132)	0.0018000350124242881
  (0, 42161)	0.00351579337027075
  (0, 2786)	0.002291525876047508
  (0, 38523)	0.004424279896694256
  (0, 49538)	0.0021226517551796165
  (0, 45167)	0.004799839237446008
  (0, 45168)	0.004799839237446008
  (0, 12411)	0.003951131083974518
  (0, 1353)	0.004799839237446008
  (0, 22139)	0.0020021909161804527
  (0, 37456)	0.003066095007628514
  (0, 2771)	0.002291525876047508
  (0, 1086)	0.0029974529947649152
  (0, 43138)	0.003066095007628514
  (0, 2116)	0.003951131083974518
  (0, 354)	0.002714595287184984
  (0, 363)	0.0021755943531760404
  (0, 1007)	0.0026670852167992604
  (0, 527)	0.002621893654013163
  (0, 2544)	0.0019159665352957555
  (0, 1162)	0.003066095007628514
  (0, 25976)	0.0036394759283525442
  :	:
  (129, 32182)	0.019182853946552503
  (129, 13207)	0.001370203853325179
  (129, 10819)	0.002811189108572395
  (129, 40026)	0.012559857008466168
  (129, 

In [8]:
# A: Let's transform the sparse matrix to a pandas dataframe, then save it to a file.  
import pandas as pd

m = transformed_documents.tocoo()
tuples = zip(m.row, m.col, m.data)
df1 = pd.DataFrame(tuples, columns =['docid', 'termid', 'score'])
# print(df1)

df2 = pd.DataFrame(vectorizer.get_feature_names_out(), columns = ['term'])
df2['termid'] = df2.index
# print(df2)

df3 = df1.merge(df2, on='termid')
df3 = df3.sort_values(['docid', 'score'], ascending=[True, False])
print(df3)

print(all_txt_files[0], '\n', df3[df3['docid'] == 0])
df3.to_csv('docvecs.csv', index=False)

        docid  termid     score        term
45731       0   41217  0.796370       pratt
46291       0   29708  0.260721       james
46276       0   44505  0.183921    religion
46082       0   44509  0.178289   religious
34894       0    9441  0.153595     brébeuf
...       ...     ...       ...         ...
40070     129   22449  0.001336   following
6396      129   57642  0.001303        york
9561      129   14323  0.001303    creation
157199    129   18905  0.001303  electronic
36323     129   41178  0.001293   practices

[226354 rows x 4 columns]
txts/03_Jensen_Its-not-Personal_WJS_Vol13_No-2_Fall-2017_pp-140-166.txt 
        docid  termid     score        term
45731      0   41217  0.796370       pratt
46291      0   29708  0.260721       james
46276      0   44505  0.183921    religion
46082      0   44509  0.178289   religious
34894      0    9441  0.153595     brébeuf
...      ...     ...       ...         ...
34457      0   42661  0.001372  publishing
9479       0   14323  0.001